In [1]:
import os
import numpy as np
import PIL.Image
import random
import string
import shutil
from tqdm import tqdm
import cv2
import torch
from utils.general import scale_coords, non_max_suppression, xyxy2xywh, xywhn2xyxy, xyxy2xywhn

### Get anchor of the model

In [69]:
def get_anchor():
    from models.experimental import attempt_load  # scoped to avoid circular import
    w = '/Users/zhenyu/Desktop/yolov5x6#22/weights/best.pt'
    model_pt = torch.jit.load(w) if 'torchscript' in w else attempt_load(w)
    print('Anchors are:\n {}'.format(model_pt.model[-1].anchors))
    print(model_pt.model[-1].grid[0].shape)
    print(model_pt.model[-1].grid[1].shape)
    print(model_pt.model[-1].grid[2].shape)
    print(model_pt.model[-1].grid[3].shape)
get_anchor()

Fusing layers... 
Model Summary: 574 layers, 139980484 parameters, 0 gradients, 208.1 GFLOPs


Anchors are:
 tensor([[[ 2.37500,  3.37500],
         [ 5.50000,  5.00000],
         [ 4.75000, 11.75000]],

        [[ 6.00000,  4.25000],
         [ 5.37500,  9.50000],
         [11.25000,  8.56250]],

        [[ 4.37500,  9.40625],
         [ 9.46875,  8.25000],
         [ 7.43750, 16.93750]],

        [[ 6.81250,  9.60938],
         [11.54688,  5.93750],
         [14.45312, 12.37500]]])
torch.Size([1, 3, 248, 184, 2])
torch.Size([1, 3, 124, 92, 2])
torch.Size([1, 3, 62, 46, 2])
torch.Size([1, 3, 31, 23, 2])


### Crop image to 256*256 with screw at the center

In [18]:
# crop images to 256*256 with screw at the center
IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/NG'
OUT_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/Cropped/11111/'

i=0
for folder in [each for each in os.listdir(IMG_FOLDER) if not each.startswith('.') and each.startswith('3')]:
    for img in [img for img in os.listdir(os.path.join(IMG_FOLDER, folder)) if img.endswith('.jpg')]:
        with open(os.path.join(IMG_FOLDER, folder, '{}.txt'.format(img[:-4])), 'r') as f:
            lines = f.read().splitlines()
        image = PIL.Image.open(os.path.join(IMG_FOLDER, folder, img))
        for line in lines:
            x, y = [float(num) for num in line.split(' ')[1:3]]
            x = int(x*3024)
            y = int(y*4032)
            cropped = image.crop((x-128, y-128, x+128, y+128))
            name = OUT_FOLDER + ''.join(random.choices(string.ascii_uppercase + string.digits, k=10)) + '.jpg'
            cropped.save(name)
            i+=1
            if i >= 150:
                raise NameError('TOOOOO MANYYYYY')

NameError: TOOOOO MANYYYYY

### Remove cropped images without json and rename the jpg&json

In [195]:
# get rid of cropped images without json and rename the jpg&json

IMG_FOLDER = '/Users/zhenyu/Desktop/1111_Zhenyu/'
JSON_FOLDER = IMG_FOLDER
i=0
for img in [file for file in os.listdir(IMG_FOLDER) if file.endswith('.jpg')]:
    json = '{}.json'.format(img[:-4])
    if os.path.exists(os.path.join(JSON_FOLDER, json)):
        os.rename(os.path.join(IMG_FOLDER, img), os.path.join(IMG_FOLDER, '{}.jpg'.format(i)))
        os.rename(os.path.join(JSON_FOLDER, json), os.path.join(JSON_FOLDER, '{}.json'.format(i)))
        i += 1
    else:
        os.remove(os.path.join(IMG_FOLDER, img))
        print('File: {} removed!'.format(img))

File: 11111_20_11.jpg removed!
File: 11111_1_16.jpg removed!
File: 11111_20_16.jpg removed!
File: 11111_1_39.jpg removed!
File: 11111_0_1.jpg removed!
File: 11111_0_22.jpg removed!
File: 11111_21_30.jpg removed!
File: 11111_21_31.jpg removed!
File: 11111_0_7.jpg removed!
File: 11111_0_18.jpg removed!
File: 11111_21_34.jpg removed!
File: 11111_21_36.jpg removed!
File: 11111_21_37.jpg removed!
File: 11111_20_46.jpg removed!
File: 11111_1_47.jpg removed!
File: 11111_1_35.jpg removed!
File: 11111_20_19.jpg removed!
File: 11111_20_32.jpg removed!
File: 11111_20_5.jpg removed!
File: 11111_20_33.jpg removed!


### Select OK images randomly to destination

In [36]:
# randomly select OK images to destination and rename

IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/OK/'
OUT_FOLDER = '/Users/zhenyu/Desktop/OK_Images/n'

img_list = []
for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        if name.endswith('.jpg'):
            img_list.append(os.path.join(root, name))
rand_img_list = random.sample(img_list, 500)
for file in tqdm(rand_img_list):
    shutil.copyfile(file, os.path.join(OUT_FOLDER, file.split('/')[-1]))

100%|██████████| 500/500 [00:05<00:00, 96.21it/s] 


### Move labelled data (from LabelImg) to test folder

In [3]:
# move labelled data to test folder

IMG_FOLDER = '/Users/zhenyu/Downloads/NG'
TEST_FOLDER = '/Users/zhenyu/Desktop/validation/'

for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        label = os.path.join(root, '{}.txt'.format(name[:-4]))
        if name.endswith('.jpg') and os.path.exists(label):
            shutil.copyfile(os.path.join(root, name), os.path.join(TEST_FOLDER, 'images/test', name))
            shutil.copyfile(label, os.path.join(TEST_FOLDER, 'labels/test', '{}.txt'.format(name[:-4])))

### Move synthesized data to test folder

In [ ]:
# move labelled data to test folder

IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/Artificial_Screw_Generation/ImageGeneration20211217/Station1/Batch1/Images/'
LABEL_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/Artificial_Screw_Generation/ImageGeneration20211217/Station1/Batch1/Yolo_Format/'

IMG_DST = '/Users/zhenyu/Desktop/dataset_1228/images/train/'
LABEL_DST = '/Users/zhenyu/Desktop/dataset_1228/labels/train/'

IMG_LIST = []
LABEL_LIST = []

for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        if name.endswith('.jpg'):
            IMG_LIST.append(name[:-4])
        if name.endswith('.txt'):
            LABEL_LIST.append(name[:-4])
for name in tqdm(IMG_LIST):
    if name in LABEL_LIST:
        shutil.copyfile(os.path.join(IMG_FOLDER, '{}.jpg'.format(name)), os.path.join(IMG_DST, '{}.jpg'.format(name)))
        shutil.copyfile(os.path.join(LABEL_FOLDER, '{}.txt'.format(name)), os.path.join(LABEL_DST, '{}.txt'.format(name)))

### Remove images without labels

In [35]:
# get rid of images without labels

IMG_FOLDER = '/Users/zhenyu/Desktop/dataset_1228/images/train'
LABEL_FOLDER = '/Users/zhenyu/Desktop/dataset_1228/labels/train'
i=0
for img in [file for file in os.listdir(IMG_FOLDER) if file.endswith('.jpg')]:
    label = '{}.txt'.format(img[:-4])
    if os.path.exists(os.path.join(LABEL_FOLDER, label)):
#         os.rename(os.path.join(IMG_FOLDER, img), os.path.join(IMG_FOLDER, '{}.jpg'.format(i)))
#         os.rename(os.path.join(LABEL_FOLDER, label), os.path.join(LABEL_FOLDER, '{}.txt'.format(i)))
#         i += 1
        continue
    else:
        os.remove(os.path.join(IMG_FOLDER, img))
        print('File: {} removed!'.format(img))

File: Raw_2021-12-18 15:59:07.866141.jpg removed!
File: Raw_2021-12-18 15:59:09.380036.jpg removed!
File: Raw_2021-12-18 15:58:52.539909.jpg removed!
File: Raw_2021-12-18 15:59:06.662317.jpg removed!
File: Raw_2021-12-18 15:58:50.971603.jpg removed!
File: Raw_2021-12-18 15:58:44.597078.jpg removed!
File: Raw_2021-12-18 15:58:45.140753.jpg removed!
File: Raw_2021-12-18 15:58:45.507688.jpg removed!
File: Raw_2021-12-18 15:58:45.494592.jpg removed!


### Draw Bounding Box

In [ ]:
# draw bounding box

IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/NG/33333_20211209111350/'
LABEL_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/NG/33333_20211209111350/'

def draw_bb(image, label, color=(0,0,255)):
    img = cv2.imread(IMG_FOLDER+image)
    
    img_h, img_w = img.shape[:2]

    for box in label:
        x, y, w, h = box[1:]

        x_min = int((x-w/2)*img_w)
        y_min = int((y-h/2)*img_h)
        x_max = int((x+w/2)*img_w)
        y_max = int((y+h/2)*img_h)
        rec_img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=2)
    cv2.imwrite(IMG_FOLDER+image[:-4]+'_test.jpg', rec_img)

for image in [f for f in os.listdir(IMG_FOLDER) if f.endswith('.jpg')]:
    try:
        with open(LABEL_FOLDER+image[:-4]+'.txt') as f:
            label = [list(map(float, line.rstrip().split())) for line in f]
        draw_bb(image, label)
    except FileNotFoundError:
        print('File Not Found')
        pass

### Hightlight overkills&escapes

In [201]:
# highlight all overkilled and escaped objects

IMG_FOLDER = '/Users/zhenyu/Desktop/dataset_1230/images/test'
LABEL_FOLDER = '/Users/zhenyu/Desktop/dataset_1230/labels/test'
PRED_FOLDER = '/Users/zhenyu/Desktop/pred'
OUT_FOLDER = '/Users/zhenyu/Desktop/iou'

def iou_calc(box1, box2, eps=1e-7):
    
    box2 = box2.T
    
    b1_x1, b1_x2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
    b1_y1, b1_y2 = box1[2] - box1[4] / 2, box1[2] + box1[4] / 2
    b2_x1, b2_x2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2
    b2_y1, b2_y2 = box2[2] - box2[4] / 2, box2[2] + box2[4] / 2
    
    inter = (np.minimum(b1_x2, b2_x2) - np.maximum(b1_x1, b2_x1)).clip(0) * \
            (np.minimum(b1_y2, b2_y2) - np.maximum(b1_y1, b2_y1)).clip(0)

    # Union Area
    w1, h1 = box1[3], box1[4] + eps
    w2, h2 = box2[3], box2[4] + eps
    union = w1 * h1 + w2 * h2 - inter + eps

    iou = inter / union
    return iou

def draw_bb_iou(img, label, color=(0,0,255), text=''):
    
    img_h, img_w = img.shape[:2]

    for box in label:
        x, y, w, h = box[1:]

        x_min = int((x-w/2)*img_w)
        y_min = int((y-h/2)*img_h)
        x_max = int((x+w/2)*img_w)
        y_max = int((y+h/2)*img_h)
        img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=4)
        cv2.putText(img, text, (x_min, y_min-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 4)
    
    return img

n_label = 0
n_pred = 0
n_FP = 0
n_FN = 0
# TP = []
# FN = []

for image in tqdm([img for img in os.listdir(IMG_FOLDER) if img.endswith('.jpg')]):
    try:
        with open(os.path.join(LABEL_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            label = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        label = []
    try:
        with open(os.path.join(PRED_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            pred = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        pred = []
    label.sort()
    pred.sort()
    label = np.array(label)
    n_label += label.shape[0]
    pred = np.array(pred)
    n_pred += pred.shape[0]
    index_label_list = []
    
#     img = cv2.imread(os.path.join(IMG_FOLDER, image))
    for index_label, value in enumerate(label):
        iou = iou_calc(value, pred[:-1])
        try:
            iou_max, index_pred = torch.max(torch.tensor(iou), 0)
        except IndexError:
            continue
        if iou_max >= 0.5:
#             img = draw_bb_iou(img, [value], color=(0,255,0))
#             TP.append(pred[index_pred][5])
            pred = np.delete(pred, index_pred, 0)
            index_label_list.append(index_label)
        elif iou_max >= 0.3:
#             img = draw_bb_iou(img, [value], color=(255,0,0), text='Low IOU')
#             TP.append(pred[index_pred][5])
            pred = np.delete(pred, index_pred, 0)
            index_label_list.append(index_label)
    label = np.delete(label, index_label_list, 0)
    
    n_FN += label.shape[0]
    n_FP += pred.shape[0]
#     for each in pred:
#         FN.append(each[5])
    
#     img = draw_bb_iou(img, label, color=(0,0,255), text='Escape')
#     img = draw_bb_iou(img, pred, color=(0,0,0), text='Overkill')
            
#     cv2.imwrite(os.path.join(OUT_FOLDER, '{}_iou.jpg'.format(image[:-4])), img)
print('Precision is {:%}'.format((n_pred-n_FP)/n_pred))
print('Recall is {:%}'.format((n_pred-n_FP)/n_label))
print('Escape rate is {:%}'.format(1-(n_pred-n_FP)/n_label))
print('Overkill rate is {:%}'.format((n_FP)/n_label))

100%|██████████| 718/718 [00:01<00:00, 374.91it/s]

Precision is 97.067795%
Recall is 97.307161%
Escape rate is 2.692839%
Overkill rate is 2.939436%


In [184]:
# highlight all overkilled and escaped objects

IMG_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/Overkill/Raw/FOV3_136'
LABEL_FOLDER = '/Users/zhenyu/Library/CloudStorage/Box-Box/MLProject:IphoneAOI/datasets/After T-Cowling/Overkill/Raw/FOV3_136'
PRED_FOLDER = '/Users/zhenyu/Desktop/FOV3_136'
OUT_FOLDER = '/Users/zhenyu/Desktop/iou'

def iou_calc(box1, box2, eps=1e-7):
    
    box2 = box2.T
    
    b1_x1, b1_x2 = box1[1] - box1[3] / 2, box1[1] + box1[3] / 2
    b1_y1, b1_y2 = box1[2] - box1[4] / 2, box1[2] + box1[4] / 2
    b2_x1, b2_x2 = box2[1] - box2[3] / 2, box2[1] + box2[3] / 2
    b2_y1, b2_y2 = box2[2] - box2[4] / 2, box2[2] + box2[4] / 2
    
    inter = (np.minimum(b1_x2, b2_x2) - np.maximum(b1_x1, b2_x1)).clip(0) * \
            (np.minimum(b1_y2, b2_y2) - np.maximum(b1_y1, b2_y1)).clip(0)

    # Union Area
    w1, h1 = box1[3], box1[4] + eps
    w2, h2 = box2[3], box2[4] + eps
    union = w1 * h1 + w2 * h2 - inter + eps

    iou = inter / union
    return iou

def draw_bb_iou(img, label, color=(0,0,255), text=''):
    
    img_h, img_w = img.shape[:2]

    for box in label:
        x, y, w, h = box[1:]

        x_min = int((x-w/2)*img_w)
        y_min = int((y-h/2)*img_h)
        x_max = int((x+w/2)*img_w)
        y_max = int((y+h/2)*img_h)
        img = cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=4)
        cv2.putText(img, text, (x_min, y_min-10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 4)
    
    return img

n_label = 0
n_pred = 0
n_FP = 0
n_FN = 0
# TP = []
# FN = []
MESA = []

for image in tqdm([img for img in os.listdir(IMG_FOLDER) if img.endswith('.jpg')]):
    try:
        with open(os.path.join(LABEL_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            label = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        label = []
    try:
        with open(os.path.join(PRED_FOLDER, '{}.txt'.format(image[:-4]))) as f:
            pred = [list(map(float, line.rstrip().split())) for line in f]
    except FileNotFoundError:
        pred = []

    label.sort()
    pred.sort()
    label = np.array(label)
    n_label += label.shape[0]
    pred = np.array(pred)
    n_pred += pred.shape[0]
    index_label_list = []
    
    for index_label, value in enumerate(pred):
        if 0.1256<value[1]<0.33 and 0.62<value[2]<0.7565:
            MESA.append(value[5])
    
#     img = cv2.imread(os.path.join(IMG_FOLDER, image))
#     for index_label, value in enumerate(label):
#         iou = iou_calc(value, pred[:-1])
#         try:
#             iou_max, index_pred = torch.max(torch.tensor(iou), 0)
#         except IndexError:
#             continue
#         if iou_max >= 0.5:
# #             img = draw_bb_iou(img, [value], color=(0,255,0))
#             TP.append(pred[index_pred][5])
#             pred = np.delete(pred, index_pred, 0)
#             index_label_list.append(index_label)
#         elif iou_max >= 0.3:
# #             img = draw_bb_iou(img, [value], color=(255,0,0), text='Low IOU')
#             TP.append(pred[index_pred][5])
#             pred = np.delete(pred, index_pred, 0)
#             index_label_list.append(index_label)
#     label = np.delete(label, index_label_list, 0)
    
#     n_FN += label.shape[0]
#     n_FP += pred.shape[0]
#     for each in pred:
#         FN.append(each[5])
    
# #     img = draw_bb_iou(img, label, color=(0,0,255), text='Escape')
# #     img = draw_bb_iou(img, pred, color=(0,0,0), text='Overkill')
            
# #     cv2.imwrite(os.path.join(OUT_FOLDER, '{}_iou.jpg'.format(image[:-4])), img)
# print('Precision is {:%}'.format((n_pred-n_FP)/n_pred))
# print('Recall is {:%}'.format((n_pred-n_FP)/n_label))
# print('Escape rate is {:%}'.format(1-(n_pred-n_FP)/n_label))
# print('Overkill rate is {:%}'.format((n_FP)/n_label))

100%|██████████| 136/136 [00:00<00:00, 2702.27it/s]


In [188]:
import seaborn as sns
df = pd.DataFrame([FN, TP, MESA]).transpose()
df = df.rename(columns={0:'FN', 1:'TP', 2:'MESA'})
df['id'] = df.index
df = pd.melt(df, value_vars=['FN', 'TP', 'MESA'])
sns.displot(df, x='value', hue='variable', kind='hist', height=10)
plt.savefig('/Users/zhenyu/Desktop/hist.png')

### Remove Images with empty label

In [208]:
IMG_FOLDER = '/Users/zhenyu/Desktop/OK_label'

for root, dirs, files in os.walk(IMG_FOLDER):
    for name in files:
        if name.endswith('.jpg'):
            txt_name = '{}.txt'.format(name[:-4])
            try:
                with open(os.path.join(root,txt_name)) as f:
                    label = [list(map(float, line.rstrip().split())) for line in f]
                if not label:
                    os.remove(os.path.join(root,txt_name))
                    os.remove(os.path.join(root,name))
            except FileNotFoundError:
                os.remove(os.path.join(root,name))

### Replace labels 1 with 0

In [23]:
LABEL_FOLDER = '/Users/zhenyu/Desktop/dataset_1228/labels/test/'

for file in [file for file in os.listdir(LABEL_FOLDER) if file.endswith('txt')]:
    with open(os.path.join(LABEL_FOLDER, file)) as f:
        label = [list(map(float, line.rstrip().split())) for line in f]
    for each in label:
        if each[0] == 1:
            print(file)
            break

_E36DFDED-3A42-4BAC-9460-C63E7D6173D4.txt
_4AEB612E-DA7C-4EE5-82D8-B59A2DBCC605.txt
_85C459D4-CD8C-4701-9BC5-C83127D49CB9.txt
_0E064716-EE76-4373-9857-9EFCDC536E0E.txt
_3B9CAB7B-B975-476B-9B45-4BBE18F6DCCF.txt


In [25]:
LABEL_FOLDER = '/Users/zhenyu/Desktop/dataset_1228/labels/test/'

for file in [file for file in os.listdir(LABEL_FOLDER) if file.endswith('txt')]:
    with open(os.path.join(LABEL_FOLDER, file)) as f:
        label = [list(map(float, line.rstrip().split())) for line in f]
    for each in label:
        if each[0] == 1:
            print(file)
            break

In [59]:
name_list = name.split('\n')

In [199]:
def condition(x):
    return x > 0.7
1- sum(condition(x) for x in MESA)/len(MESA)

0.948051948051948

### Unzip Insight data and move to destination

In [215]:
import tarfile
FOLDER_PATH = '/Users/zhenyu/Downloads/Data8'
DST_PATH = '/Users/zhenyu/Desktop/dataset_0111/images/test'

for root, dirs, files in os.walk(FOLDER_PATH):
    for zipfile in tqdm(files):
        if zipfile.endswith('.tgz'):
            tar = tarfile.open(os.path.join(root, zipfile), "r:gz")
            tar.extractall(path=root)
            tar.close()
for root, dirs, files in tqdm(os.walk(FOLDER_PATH)):
    for file in files:
        if file == '1.jpg' or file == '2.jpg':
            shutil.copyfile(os.path.join(root,file), os.path.join(DST_PATH,str(i)+'.jpg'))
            i+=1

100%|██████████| 250/250 [00:35<00:00,  7.09it/s]
496it [00:05, 85.64it/s]
